In [2]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy.fftpack import fft
from scipy.ndimage.filters import gaussian_filter

from sklearn.metrics import f1_score
# import tensorflow_addons as ta

plt.style.use('seaborn')

/Users/nbaudis/.anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nbaudis/.anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nbaudis/.anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/nbaudis/.anaconda3/envs/ml/lib/python3.7/s

### Import Data

In [3]:
from utils.data import *

In [4]:
x_train, y_train = load_training_data()

In [5]:
print(f'Shape of x_train: {x_train.shape}')
print(f'Shape of y_train: {y_train.shape}')

Shape of x_train: (45360, 512, 3)
Shape of y_train: (45360,)


### Extract Features with PyEEG

In [5]:
import pyeeg

In [108]:
fisher_info = np.apply_along_axis(pyeeg.fisher_info, axis=1, 
                                  arr=x_train, Tau=4, DE=10)

KeyboardInterrupt: 

In [21]:
# Too slow
ap_entropy = np.apply_along_axis(pyeeg.ap_entropy, axis=1, 
                                 arr=x_train[:100], M=2, R=0.5)

In [88]:
def extract_data(x):
    # Generate training set with extracted features
    # Takes long, save to numpy file afterwards
    pfd = np.apply_along_axis(pyeeg.pfd, axis=1, arr=x)

    hfd = np.apply_along_axis(pyeeg.hfd, axis=1, arr=x, Kmax=5)

    hjorth = np.apply_along_axis(pyeeg.hjorth, axis=1, arr=x)
    hjorth = hjorth.reshape(x.shape[0], 2 * x.shape[2])

    hurst = np.apply_along_axis(pyeeg.hurst, axis=1, arr=x)
    hurst = np.nan_to_num(hurst)

    dfa = np.apply_along_axis(pyeeg.dfa, axis=1, arr=x)

    spectral_entropy = np.apply_along_axis(pyeeg.spectral_entropy, axis=1, 
                                 arr=x, Band=[0.5,4,7,12,30,64], Fs=128)

    std = np.apply_along_axis(np.std, axis=1, arr=x)
    
    x_feat = np.concatenate((dfa, hfd, hjorth, pfd, spectral_entropy, hurst, std), axis=1)

x_train_feat = extract_training_data(x_train)
np.save('data/x_train_feat', x_train_feat)
print(x_train_feat.shape)

(45360, 24)


In [6]:
x_train_feat = np.load('data/x_train_feat.npy')
print(x_train_feat.shape)

(45360, 24)


### Build Deep Neural Network

In [7]:
# Generate weights for training, since the classes are imbalanced
class_weight = {}

count_0, count_1, count_2 = np.bincount(y_train)
total = count_0 + count_1 + count_2

weight_0 = (1 / count_0) * total
weight_1 = (1 / count_1) * total
weight_2 = (1 / count_2) * total

class_weight[0] = weight_0
class_weight[1] = weight_1
class_weight[2] = weight_2

In [8]:
# Split data for validation
x_train_feat, x_val_feat, y_train, y_val = train_test_split(x_train_feat, y_train, test_size=0.2, random_state=42)

In [66]:
model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(1, 18)),
    keras.layers.Dense(2048, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    # keras.layers.Dense(128, activation='relu'),
    # keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='Adam', 
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])

In [14]:
model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(1, 18)),
    keras.layers.Dense(512, activation='sigmoid'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(512, activation='sigmoid'),
    keras.layers.Dropout(0.1),
#     keras.layers.Dense(128, activation='sigmoid'),
#     keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='Adam', 
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])

In [15]:
history = model.fit(x_train_feat, y_train, class_weight=class_weight, epochs=15, validation_data=(x_val_feat, y_val))

Train on 36288 samples, validate on 9072 samples
Epoch 1/15
36288/36288 [==============================] - 3s 93us/sample - loss: 2.4870 - acc: 0.6542 - val_loss: 0.6479 - val_acc: 0.7336
Epoch 2/15
36288/36288 [==============================] - 3s 92us/sample - loss: 2.3854 - acc: 0.6604 - val_loss: 0.8376 - val_acc: 0.5209
Epoch 3/15
36288/36288 [==============================] - 3s 93us/sample - loss: 2.3650 - acc: 0.6620 - val_loss: 0.6857 - val_acc: 0.6596
Epoch 4/15
36288/36288 [==============================] - 3s 94us/sample - loss: 2.3446 - acc: 0.6672 - val_loss: 0.7098 - val_acc: 0.6569
Epoch 5/15
36288/36288 [==============================] - 5s 136us/sample - loss: 2.3069 - acc: 0.6742 - val_loss: 0.6682 - val_acc: 0.7007
Epoch 6/15
36288/36288 [==============================] - 7s 190us/sample - loss: 2.2989 - acc: 0.6792 - val_loss: 0.6596 - val_acc: 0.7002
Epoch 7/15
36288/36288 [==============================] - 4s 113us/sample - loss: 2.2256 - acc: 0.7001 - val_loss: 

In [12]:
train_f1 = f1_score(y_train, np.argmax(model.predict(x_train_feat), axis=1), average='macro')
print(f'F1 score on training data: {train_f1:.4f}')

F1 score on training data: 0.6943


In [13]:
val_f1 = f1_score(y_val, np.argmax(model.predict(x_val_feat), axis=1), average='macro')
print(f'F1 score on validation data: {val_f1:.4f}')

F1 score on validation data: 0.6916


### Evaluate on Test Data

In [14]:
x_test = load_testing_data()

In [42]:
# Test set with extracted features
x_test_feat = extract_data(x_test)
np.save('data/x_test_feat', x_test_feat)
print(x_test_feat.shape)

(19440, 24)


In [43]:
x_test_feat = np.load('data/x_test_feat.npy')

In [106]:
y_test_pred = model.predict(x_test_feat)
generate_submission(y_test_pred, 'DeepNN_Pred')

In [107]:
print(y_test_pred.argmax(axis=1))

[1 1 2 ... 0 0 0]


In [16]:
df_sub = pd.read_csv('submissions/DeepNN_Pred_F758.csv')

In [17]:
print(df_sub)

          Id  y
0          0  1
1          1  1
2          2  2
3          3  1
4          4  2
...      ... ..
19435  19435  0
19436  19436  0
19437  19437  0
19438  19438  0
19439  19439  0

[19440 rows x 2 columns]


In [18]:
df_sub.y += 1

In [19]:
print(df_sub)

          Id  y
0          0  2
1          1  2
2          2  3
3          3  2
4          4  3
...      ... ..
19435  19435  1
19436  19436  1
19437  19437  1
19438  19438  1
19439  19439  1

[19440 rows x 2 columns]


In [21]:
df_sub.to_csv('submissions/DeepNN_Pred_F758_new.csv', index=False)